# Модуль 1.4 — Локальные модели в Colab

**Цель:** запустить локальную LLM на GPU без внешнего API и понять ограничения.

**Что сделаем:**
- проверим доступность GPU
- установим `transformers`
- запустим небольшую русскоязычную модель
- разберем, когда локальные модели выгодны

In [ ]:
%pip -q install -U transformers accelerate sentencepiece

## Проверка GPU

Colab с GPU ускоряет инференс. Если GPU недоступен, все будет работать, но медленнее.

In [ ]:
import torch

print("CUDA доступна:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

## Загрузка локальной модели

Для обучения берем небольшую русскоязычную модель. Она быстрее скачивается и подходит для демонстраций.

**Почему не Llama‑3 в Colab:**
- крупные модели требуют много VRAM и времени загрузки
- в Colab часто удобнее начинать с компактных моделей

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

local_model_name = "ai-forever/rugpt3small_based_on_gpt2"

tokenizer = AutoTokenizer.from_pretrained(local_model_name)
model = AutoModelForCausalLM.from_pretrained(
    local_model_name,
    torch_dtype="auto",
    device_map="auto",
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

prompt = "Сгенерируй 3 варианта заголовка статьи про практическое применение LangChain."
result = generator(
    prompt,
    max_new_tokens=80,
    do_sample=True,
    temperature=0.7,
)

print(result[0]["generated_text"])

## Практические рекомендации

- Для демо берите компактные модели (до 1–3B), чтобы не упираться в память.
- Снижайте `temperature`, если нужно стабильнее следовать формату.
- Для ускорения используйте GPU и `device_map="auto"`.
- Локальные модели хороши для офлайн‑экспериментов и приватных данных.
- Для продакшена обычно нужен более мощный сервер или оптимизация (квантование).